In [2]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_new, kernels_reparametrization_space as kernels_repar_space
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

ImportError: cannot import name 'kernels_new' from 'GEMS_TCO' (/Users/joonwonlee/Documents/GEMS_TCO-1/src/GEMS_TCO/__init__.py)

Load monthly data

In [2]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


Load daily data applying max-min ordering

In [3]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []        

for day_index in range(31):
  
    hour_start_index = day_index * 8
    #hour_end_index = (day_index + 1) * 8
    hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm=ord_mm,  
        dtype=torch.float 
    )
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
#print(daily_hourly_maps[0])

torch.Size([18126, 4])


Hyper parameters

In [4]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=150

Adams Optimization Full batch + plateau scheduler (5)

lr 0.01 use plateau scheduler for 1120 epochs 400, pateince 5 factor 0.5

In [ ]:

# --- End Placeholders ---

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                
    init_phi1 = init_sigmasq * init_phi2            
    init_phi3 = (init_range_lon / init_range_lat)**2  
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 4-parameter spatial-only list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ),
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )
    ]

    # --- Define learning rates and parameter groups ---
    # 💥 REVISED: Set lr_fast from the global lr
    lr_fast = lr 
    fast_indices = [0, 1, 2, 3] # All parameters
    
    param_groups = [
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    # --- Print Job Info ---
    # Assuming lat_lon_resolution, v, mm_cond_number are defined globally
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Instantiate the Correct Class ---
    model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 REVISED: Call the Optimizer Method with Plateau settings ---
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-5, 
            scheduler_type='plateau', # Explicitly set to plateau
            patience=patience,        # Pass patience
            factor=factor             # Pass factor
        )

    # --- Call the Correct Fit Method ---
    out, epoch_ran = model_instance.fit_vecc_scheduler_fullbatch(
            params_list,
            optimizer,
            scheduler, 
            model_instance.matern_cov_SPATIAL_log_reparam, 
            epochs=epochs
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results: {out}")

L-BFGS Optimization

In [5]:
# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    
    # Define initial *model* parameters
    init_sigmasq   = 25.0 #15
    init_range_lat = 1.66 
    init_range_lon = 1.7 
    init_nugget    = 1.5
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # [1] phi2 = 1 / range_lon
    init_phi1 = init_sigmasq * init_phi2            # [0] phi1 = sigmasq * phi2 = sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # [2] phi3 = (range_lon / range_lat)^2
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 4-parameter spatial-only list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [3] log(nugget)
    ]

    # --- Print Job Info ---
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
    
    # --- 💥 REVISED: Instantiate the LBFGS Class ---
    model_instance = kernels_repar_space.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 REVISED: Call the LBFGS Optimizer Method ---
    # L-BFGS uses a different set of parameters and doesn't return a scheduler.
    # We pass the list of parameters directly, which set_optimizer will wrap.
    optimizer = model_instance.set_optimizer(
            params_list,     
            lr=1.0,            # L-BFGS often uses lr=1.0 for its line search
            max_iter=20        # Max iterations *per step*
        )

    # --- 💥 REVISED: Call the LBFGS Fit Method ---
    # This method takes 'max_steps' (epochs) and no scheduler.
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer,
            model_instance.matern_cov_SPATIAL_log_reparam, # Pass the spatial-only method
            max_steps=50 # Total number of L-BFGS steps
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {steps_ran+1} steps.")
    print(f"Day {day_idx+1} final results: {out}")


--- Starting Day 1 (2024-07-1) ---
Data size per day: 17854, smooth: 0.5
mm_cond_number: 8,
initial parameters: 
 [tensor([2.6882], dtype=torch.float64, requires_grad=True), tensor([-0.5306], dtype=torch.float64, requires_grad=True), tensor([0.0476], dtype=torch.float64, requires_grad=True), tensor([0.4055], dtype=torch.float64, requires_grad=True)]
--- Starting L-BFGS Optimization ---
--- Step 1/50 / Loss: 1.346422 ---
  Param 0: Value=3.6858, Grad=0.002046299528253265
  Param 1: Value=0.7876, Grad=-0.0016276695741195898
  Param 2: Value=0.4180, Grad=0.0020702780041881633
  Param 3: Value=0.4425, Grad=0.0027643616378248688
  Max Abs Grad: 2.764362e-03
------------------------------
--- Step 2/50 / Loss: 1.260479 ---
  Param 0: Value=3.8729, Grad=1.4710556778979147e-06
  Param 1: Value=1.2729, Grad=-2.1735015135278492e-07
  Param 2: Value=0.3052, Grad=2.0061605042351713e-07
  Param 3: Value=0.2217, Grad=5.48954400494314e-07
  Max Abs Grad: 1.471056e-06
------------------------------



# Result

In [11]:
def cal(a):
    day_indices = [0] 
    for day_idx in day_indices:  

        daily_hourly_map = daily_hourly_maps[day_idx]
        daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

        # Convert initial parameters to a list of 1-element tensors
        params_list = [
            torch.tensor([val], dtype=torch.float64, requires_grad=True, device=device_str) for val in a
        ]
        
        # 💡 CRITICAL: Concatenate the list into a single tensor
        # The full_likelihood function expects a single tensor, not a list
        params_tensor = torch.cat(params_list)

        # Assuming 'kernels_repar_space' has your 'fit_vecchia_adams' class
        model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
                smooth = v,
                input_map = daily_hourly_map,
                aggregated_data = daily_aggregated_tensor,
                nns_map = nns_map,
                mm_cond_number = mm_cond_number,
                nheads = nheads
            )
        
        # 💡 Pass the single 'params_tensor' and the correct 4-parameter spatial covariance function
        bb = model_instance.full_likelihood_avg(
            params = params_tensor, 
            input_data = daily_aggregated_tensor, 
            y = daily_aggregated_tensor[:,2], 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam
        )
     
        cov_map = model_instance.cov_structure_saver(params_tensor, model_instance.matern_cov_SPATIAL_log_reparam)
        vecchia_nll = model_instance.vecchia_space_fullbatch( # Change this to your chosen Vecchia implementation
            params = params_tensor, 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam, 
            cov_map = cov_map # Assuming cov_map is precomputed or computed internally
        )
 
    return bb, vecchia_nll

device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
# adams and lfgs
#a= [3.2691297610712176, 0.6684659526552683, 0.4027674047362919, 0.8989261051578574, 6521.4909959990455]

#bb, nll_value = cal(a)
#print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()}")

def cc(n,nheads,mm):
    print(n**2 , nheads**2 + (n-nheads)*mm**3)
    return n**2 >nheads**2 + (n-nheads)*mm**3

cc(1000,300,8)

1000000 448400


True

# 18612

In [ ]:
# 18612 vecchia ( adams  match with l bfgs)

## Vecchia approximation mm 8 nnheads 300

a = [3.872930353181186, 1.2728730527962047, 0.3051836846976499, 0.22169508949125624]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [3.972930353181186, 1.2728730527962047, 0.3051836846976499, 0.22169508949125624]
 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [3.772930353181186, 1.2728730527962047, 0.3051836846976499, 0.22169508949125624]

bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")


## Debiased whittle likelihood

# 18612

a = [3.6371, 1.3191, 0.8742, 0.4129]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [3.7371, 1.3191, 0.8742, 0.4129]
 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}")

a = [3.5371, 1.3191, 0.8742, 0.4129]

bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*18162, nll_value.item()*18162}") # 17854 not 18162

Full Negative Log-Likelihood: (22927.958040292695, 22881.32979284226)
Full Negative Log-Likelihood: (22945.112672514406, 22901.431024564135)
Full Negative Log-Likelihood: (22951.00296160876, 22901.14143154865)
Full Negative Log-Likelihood: (23006.5323887555, 22962.047158892245)
Full Negative Log-Likelihood: (23012.931949116693, 22976.066939725904)
Full Negative Log-Likelihood: (23035.272549634523, 22981.05356068179)


72305 73312  

# 4560


full adams plateau / full adams scheduler 80 / l bfgs

In [ ]:
#mplateau 5 + lr 0.1 adams full batch 4560


a = [3.2813904798406806, 0.6529141404208029, 0.39520496555136536, 0.9020642550783797]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*4560, nll_value.item()*4560}")

a = [3.3813904798406806, 0.6529141404208029, 0.39520496555136536, 0.9020642550783797]
 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*4560, nll_value.item()*4560}")

a = [3.1813904798406806, 0.6529141404208029, 0.39520496555136536, 0.9020642550783797]

bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*4560, nll_value.item()*4560}")



# l bfgs
a = [3.008192996038009, 0.46428191135000707, 0.39131753527431623, 1.1231542142224333]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*4560, nll_value.item()*4560}")

a = [3.108192996038009, 0.46428191135000707, 0.39131753527431623, 1.1231542142224333]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*4560, nll_value.item()*4560}")

a = [2.908192996038009, 0.46428191135000707, 0.39131753527431623, 1.1231542142224333]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*4560, nll_value.item()*4560}")

Full Negative Log-Likelihood: (6525.265703352871, 6521.895711794799)
Full Negative Log-Likelihood: (6530.741398205831, 6527.769810700424)
Full Negative Log-Likelihood: (6526.764426877938, 6522.974295124629)
Full Negative Log-Likelihood: (6555.803170980535, 6552.803221680461)
Full Negative Log-Likelihood: (6545.497437273466, 6542.951497200811)
Full Negative Log-Likelihood: (6574.435587568947, 6570.946772134326)
Full Negative Log-Likelihood: (6523.731296642555, 6519.89079612747)
Full Negative Log-Likelihood: (6524.706108680448, 6521.245086255674)
Full Negative Log-Likelihood: (6527.921681011694, 6523.681908978209)


# 1120

epochs 200 lr 0.1 stepsize 80 nheads 300 gamma 0.5 pleateau 5 mm 8


In [ ]:
# full batch adams plateau 5, 1120 mm 8 nheads 300 gamma 0.5 step 80 lr 0.1

a = [2.586668163372335, 0.04786782666397918, 0.7218958281570125, 1.4053391894954903]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()*1120}")

a = [2.686668163372335, 0.04786782666397918, 0.7218958281570125, 1.4053391894954903] 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()*1120}")

a = [2.486668163372335, 0.04786782666397918, 0.7218958281570125, 1.4053391894954903]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()*1120}")


# LBFGS 

a = [2.5769925485448995, 0.0307829717973246, 0.7400768379343448, 1.4074311122243917]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()*1120}")

a = [2.6769925485448995, 0.0307829717973246, 0.7400768379343448, 1.4074311122243917]
 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()*1120}")

a = [2.4769925485448995, 0.0307829717973246, 0.7400768379343448, 1.4074311122243917]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()*1120}")

Full Negative Log-Likelihood: (1770.830952651946, 1768.8074920088832)
Full Negative Log-Likelihood: (1771.3446302559594, 1769.488217146782)
Full Negative Log-Likelihood: (1771.6574766693973, 1769.4531362117486)
Full Negative Log-Likelihood: (1770.80325694022, 1768.8027446852573)
Full Negative Log-Likelihood: (1771.3090202134513, 1769.472988550741)
Full Negative Log-Likelihood: (1771.6300464789101, 1769.4515538182861)
